In [5]:
import os

In [6]:
%pwd

'/home/tejas/MLProj/Thyroid-Disease-Prediction/research'

In [7]:
os.chdir('../')

In [8]:
%pwd

'/home/tejas/MLProj/Thyroid-Disease-Prediction'

In [9]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DriftMonitoringConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_path: Path
    report_path_name: str
    test_path_name: str
    target_column: str
    

In [10]:
from ThyroidProject.constants import *
from ThyroidProject.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_drift_monitoring_config(self) -> DriftMonitoringConfig:
        """
        Get the Drift Monitoring Config.

        Returns:
            DriftMonitoringConfig: The Drift Monitoring Config.

        """
        # Load the config file
        config = self.config.drift_monitoring
        # Load the params file
        params = self.params.GradientBoostedTreesLearner
        # Load the schema file
        schema = self.schema.TARGET_COLUMN

        # Create the directories
        create_directories([config.root_dir])

        # Create the DriftMonitoringConfig object
        drift_monitoring_config = DriftMonitoringConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            report_path_name=config.report_path_name,
            test_path_name=config.test_path_name,
            target_column=schema.name
        )

        return drift_monitoring_config

In [12]:
import pandas as pd
import numpy as np

from evidently import ColumnMapping

from evidently.report import Report
from evidently.metrics.base_metric import generate_column_metrics
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset, DataQualityPreset, ClassificationPreset
from evidently.metrics import *

from evidently.test_suite import TestSuite
from evidently.test_preset import DataStabilityTestPreset, NoTargetPerformanceTestPreset, MulticlassClassificationTestPreset
from evidently.tests import *

import ydf

In [15]:
class DriftMonitoring:
    def __init__(self, config: DriftMonitoringConfig):
        self.config = config
        
    def generate_drift_reports(self):
        """
        Generate the Drift Reports.

        Returns:
            None
        """
        
        # Load the data
        train_df = pd.read_csv(self.config.train_data_path)
        test_df = pd.read_csv(self.config.test_data_path)
        
        # Create the test and train features
        target = self.config.target_column
        X_train = train_df.drop(target, axis=1)
        X_test = test_df.drop(target, axis=1)
        
        # load model
        model = ydf.load_model(self.config.model_path)
        
        
        # Make predictions
        preds = model.predict(X_test)
        y_test_preds = [np.argmax(i) for i in preds]
        preds = model.predict(X_train)
        y_train_preds = [np.argmax(i) for i in preds]
        
        # print(y_test_preds , y_train_preds)
        
        # Create the column mapping for predictions
        train_df["prediction"] = y_train_preds
        test_df["prediction"] = y_test_preds
        
        # Identifying categorical and numerical columns
        cat_col = train_df.select_dtypes(include="object").columns
        num_col = train_df.select_dtypes(exclude="object").columns
        num_col = num_col.drop([self.config.target_column, 'prediction'])
        
        # Create the column mapping for evidently
        column_mapping = ColumnMapping()
        column_mapping.target = self.config.target_column
        column_mapping.prediction = "prediction"
        column_mapping.numerical_features = cat_col
        column_mapping.categorical_features = num_col
        
        # Create the report
        report = Report(metrics=[ColumnSummaryMetric(column_name='T3'),
                                 ColumnSummaryMetric(column_name='TSH'),
                                 ColumnSummaryMetric(column_name='T4U'),
                                 ColumnSummaryMetric(column_name='TT4'),
                                 generate_column_metrics(ColumnQuantileMetric, parameters={
                                     'quantile': 0.25}, columns='num'),
                                 DataDriftPreset(),
                                 TargetDriftPreset(),
                                 DataQualityPreset(),
                                 ClassificationPreset()
                                 ])


        report.run(reference_data=train_df, current_data=test_df)
        
        #save the report
        report.save_html(self.config.report_path_name)
        
        # Perform tests
        test_suite = TestSuite(tests=[
            TestNumberOfColumnsWithMissingValues(),
            TestNumberOfRowsWithMissingValues(),
            TestNumberOfConstantColumns(),
            TestNumberOfDuplicatedRows(),
            TestNumberOfDuplicatedColumns(),
            TestColumnsType(),
            TestNumberOfDriftedColumns(),
            DataStabilityTestPreset(),
            NoTargetPerformanceTestPreset(),
            MulticlassClassificationTestPreset()
        ])

        test_suite.run(reference_data=train_df, current_data=test_df)
        
        #save the test suite
        test_suite.save_html(self.config.test_path_name)
        

In [16]:
try:
    config = ConfigurationManager()
    drift_monitoring_config = config.get_drift_monitoring_config()
    drift_monitoring = DriftMonitoring(config=drift_monitoring_config)
    drift_monitoring.generate_drift_reports()
except Exception as e:
    raise e

[2024-02-03 21:36:03,343:INFO:common:yaml file: config/config.yaml loaded successfully]
[2024-02-03 21:36:03,346:INFO:common:yaml file: params.yaml loaded successfully]
[2024-02-03 21:36:03,349:INFO:common:yaml file: schema.yaml loaded successfully]
[2024-02-03 21:36:03,350:INFO:common:created directory at :artifacts]
[2024-02-03 21:36:03,351:INFO:common:created directory at :artifacts/drift_monitoring]
[2024-02-03 21:36:03,386:INFO:model_lib:Loading model from artifacts/model_trainer/model]
